# Семинар 4. Тестрование гипотез:

Краткий синопсис:
1. Разбор статьи *"How Many Random Seeds? Statistical Power Analysis in Deep Reinforcement Learning Experiments"*...
2. ...и применение выводов авторов статей на примере задачи сравнения двух моделей для предсказания рака груди (https://archive.ics.uci.edu/ml/datasets/Breast+Cancer)
3. Оценка ошибки второго рода оценки $H_0$ - что две модели имеют одну и ту же предсказательную способность

### 1. Генерация тестовых выборок с точностями (prediction accuracy) для предсказания рака груди

https://arxiv.org/abs/1806.08295

The features from the data set describe characteristics of the cell nuclei and are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. As described in [UCI Machine Learning Repository][1], the attribute informations are:

1. ID number
2. Diagnosis (M = malignant, B = benign)

3 - 32  Ten real-valued features are computed for each cell nucleus:

* a) radius (mean of distances from center to points on the perimeter)
* b) texture (standard deviation of gray-scale values)
* c) perimeter
* d) area
* e) smoothness (local variation in radius lengths)
* f) compactness (perimeter^2 / area - 1.0)
* g) concavity (severity of concave portions of the contour)
* h) concave points (number of concave portions of the contour)
* i) symmetry
* j) fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.


  [1]: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('data.csv');

print("\n \t The data frame has {0[0]} rows and {0[1]} columns. \n".format(df.shape))
df.drop(df.columns[[-1, 0]], axis=1, inplace=True)
df.head()

In [ ]:
#  count how many diagnosis are malignant (M) and how many are benign (B)
diagnosis_all = list(df.shape)[0]
diagnosis_categories = list(df['diagnosis'].value_counts())

print("\n \t The data has {} diagnosis, {} malignant and {} benign.".format(diagnosis_all, 
                                                                            diagnosis_categories[0], 
                                                                            diagnosis_categories[1]))

In [ ]:
#fancy fonts
font = {'family' : 'normal',
        'size'   : 10}
matplotlib.rc('font', **font)


# visualising the data
features_mean= list(df.columns[1:11])
plt.figure(figsize=(10,10))
sns.heatmap(df[features_mean].corr(), annot=True, square=True, cmap='coolwarm')
plt.show()

In [ ]:
color_dic = {'M':'red', 'B':'blue'}
colors = df['diagnosis'].map(lambda x: color_dic.get(x))

sm = pd.scatter_matrix(df[features_mean], c=colors, alpha=0.4, figsize=((15,15)));

plt.show()

In [ ]:
bins = 12
plt.figure(figsize=(15,15))
for i, feature in enumerate(features_mean):
    rows = int(len(features_mean)/2)
    
    plt.subplot(rows, 2, i+1)
    
    sns.distplot(df[df['diagnosis']=='M'][feature], bins=bins, color='red', label='M');
    sns.distplot(df[df['diagnosis']=='B'][feature], bins=bins, color='blue', label='B');
    
    plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
for i, feature in enumerate(features_mean):
    rows = int(len(features_mean)/2)
    
    plt.subplot(rows, 2, i+1)
    
    sns.boxplot(x='diagnosis', y=feature, data=df, palette="Set1")

plt.tight_layout()
plt.show()

In [ ]:
# training the models
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

import time
# binarasing the diagnosis
diag_map = {'M':1, 'B':0}
df['diagnosis'] = df['diagnosis'].map(diag_map)

In [ ]:
X = df.loc[:,features_mean]
y = df.loc[:, 'diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

accuracy_all = []
cvs_all = []

#### Тренируем два классификатора - Байес и KNN. 

In [ ]:
#The nearest neighbors classifier finds predefined number of training samples closest in distance to the new point, and predict the label from these.
from sklearn.neighbors import KNeighborsClassifier

start = time.time()

clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=5)

end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))
print("Execution time: {0:.5} seconds \n".format(end-start))

In [ ]:
#The Naive Bayes algorithm applies Bayes’ theorem with the assumption of independence between every pair of features.
from sklearn.naive_bayes import GaussianNB

start = time.time()

clf = GaussianNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=5)

end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))
print("Execution time: {0:.5} seconds \n".format(end-start))

#### KNN Accuracy: 93.86%, Bayes Accuracy: 94.74% 

Запомним точность с кроссвалидации на 50 фолдах для сравнения результатов работы двух алгоритмов. Как будто мы усовершенствовали KNN и хотим написать статью о новом методе, и доказываем, что он работает лучше.

In [ ]:
start = time.time()

clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=50)
data1=scores # записываем сюда 50 скоров кроссвалидации для KNN классификатора
pd.DataFrame(data1).to_csv('data1.csv')

end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))# видим что дисперсия сильно увеличилась с количеством фолдов
print("Execution time: {0:.5} seconds \n".format(end-start))

In [ ]:
scores

In [ ]:
start = time.time()

clf = GaussianNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=5)
data2=scores # записываем сюда 50 скоров кроссвалидации для Naive Bayes классификатора
pd.DataFrame(data2).to_csv('data2.csv')

end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))
print("Execution time: {0:.5} seconds \n".format(end-start))

# 2. Оценка ошибки второго рода оценки $H_0$

![](./stat_errors.png)

В статистике различаются два вида ошибок: 
  * ошибка первого рода -- когда мы отвергаем гипотезу, а она верна;
  * ошибка второго рода -- когда мы __не__ отвергаем гипотезу, а она не верна.
  
 

In [ ]:
from bootstrapped import *

Импортируем функции для подсчёта бутстрапных статистик:

  * `def bootstrap` отвечает за подсчёт интервалов с помощью бутстрапа;
  * `bootstrap_ab` производит A/B-тестирование на двух сэмплах с помощью бутстрапа;
  * `bootstrap_test` - обёртка над `bootstrap_ab`, которая дополнительно сообщает прошли ли данные тест со значимостью на уровне $\alpha$.

![Тест Уелча](http://www.statistics4u.com/fundstat_eng/img/hl_explain_welch_test.png)

$$t = \frac{x_{diff}}{\sqrt{\frac{s_1^2}{N_1} + \frac{s_2^2}{N_2}}}$$

$$v \approx \frac{\left(\frac{s_1^2}{N_1} + \frac{s_2^2}{N_2}\right)^2}{\frac{s_1^4}{N_1^2 (N_1 - 1)} + \frac{s_2^4}{N_2^2 (N_2 - 1)}}$$

#### Реализация теста Уелча
https://github.com/flowersteam/rl-difference-testing/blob/master/test_RL_difference.py

In [ ]:
from tests import bootstrap, bootstrap_ab, bootstrap_test

  * `welch_test` - реализация теста Уельча с помощью `stats.ttest_ind`;
  * `compute_beta` - функция для подсчёта вероятности ошибки II рода.
  * `empirical_false_pos_rate` - функция для ??

In [ ]:
stats.ttest_ind?

In [ ]:
from tests import compute_beta, welch_test, empirical_false_pos_rate, plot_beta

In [ ]:
# An example of AB-test with bootstrap
"""
    Imagine we own a website and think changing the color of a 'subscribe' button will improve signups. 
    One method to measure the improvement is to conduct an A/B test where we show 50% of people the old version and 50%
    of the people the new version. We can use the bootstrap to understand how much the button color improves responses 
    and give us the error bars associated with the test - this will give us lower and upper bounds on how good we should 
    expect the change to be!
"""

import numpy as np

mean = 100
stdev = 10

population = np.random.normal(loc=mean, scale=stdev, size=50000)
# take 1k 'samples' from the larger population
samples = population[:1000]

print(bootstrap(samples, stat_func=mean_))
print(bootstrap(samples, stat_func=std))

In [ ]:
# 1. just random sample. test for N random seeds will fail
data1=np.random.random_sample(50)
data2=np.random.random_sample(50)

In [ ]:
# Significance level to be used by both tests
alpha = 0.05
# Requirement in type-II error
beta_requirement = 0.2

# define the range of sample size to compute and plot beta
sample_size = range(2, 50, 2)

# define the effect size epsilon. Here we define epsilon proportionally to smaller average performance
if data1.mean() < data2.mean():
    m_smaller = data1.mean()
else:
    m_smaller = data2.mean()
epsilon = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]) * m_smaller
epsilon = epsilon.tolist()

In [ ]:
welch_test(data1, data2, alpha, tail=2)
bootstrap_test(data1, data2, alpha)
plt.show()

In [ ]:
# 2. random sample with delta
data1=np.random.random_sample(100)
data2=np.random.random_sample(100)-0.2

In [ ]:
welch_test(data1, data2, alpha, tail=2)
bootstrap_test(data1, data2, alpha)
empirical_false_pos_rate(data1, alpha)
beta = compute_beta(epsilon, sample_size, alpha, data1, data2, beta_requirement=beta_requirement)
plot_beta(beta, epsilon, sample_size, beta_requirement=beta_requirement)
plt.show()

In [ ]:
# 3. load generated data from models
path_to_data1 = 'data1.csv'
path_to_data2 = 'data2.csv'
data1 = np.asarray(pd.read_csv(path_to_data1)['0'])
data2 = np.asarray(pd.read_csv(path_to_data2)['0'])

In [ ]:
welch_test(data1, data2, alpha, tail=2)
bootstrap_test(data1, data2, alpha)
empirical_false_pos_rate(data1, alpha)
beta = compute_beta(epsilon, sample_size, alpha, data1, data2, beta_requirement=beta_requirement)
plot_beta(beta, epsilon, sample_size, beta_requirement=beta_requirement)
plt.show()

### Увеличим количество фолдов до 100

In [ ]:
start = time.time()

clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=100)
data1=scores # записываем сюда 100 скоров кроссвалидации для KNN классификатора
pd.DataFrame(data1).to_csv('data1.csv')

end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))# видим что дисперсия сильно увеличилась с количеством фолдов
print("Execution time: {0:.5} seconds \n".format(end-start))

In [ ]:
#The Naive Bayes algorithm applies Bayes’ theorem with the assumption of independence between every pair of features.
from sklearn.naive_bayes import GaussianNB

start = time.time()

clf = GaussianNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=100)
data2 =scores # записываем сюда 100 скоров кроссвалидации
pd.DataFrame(data2).to_csv('data2.csv')
end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))
print("Execution time: {0:.5} seconds \n".format(end-start))

In [ ]:
# И потом ещё стохастический градиент можно поробовать до кучи
from sklearn.linear_model import SGDClassifier

start = time.time()

clf = SGDClassifier(random_state=42)
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=100)
data2=scores # записываем сюда 100 скоров кроссвалидации для Стохастического градиента
pd.DataFrame(data2).to_csv('data2.csv')
end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("SGD Classifier Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))
print("Execution time: %s seconds \n" % "{0:.5}".format(end-start))

In [ ]:
path_to_data1 = 'data1.csv'
path_to_data2 = 'data2.csv'
data1 = np.asarray(pd.read_csv(path_to_data1)['0'])
data2 = np.asarray(pd.read_csv(path_to_data2)['0'])
welch_test(data1, data2, alpha, tail=2)
bootstrap_test(data1, data2, alpha)
empirical_false_pos_rate(data1, alpha)
beta = compute_beta(epsilon, sample_size, alpha, data1, data2, beta_requirement=beta_requirement)
plot_beta(beta, epsilon, sample_size, beta_requirement=beta_requirement)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

start = time.time()

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
scores = cross_val_score(clf, X, y, cv=100)

end = time.time()

accuracy_all.append(accuracy_score(prediction, y_test))
cvs_all.append(np.mean(scores))

print("Random Forest Accuracy: {0:.2%}".format(accuracy_score(prediction, y_test)))
print("Cross validation score: {0:.2%} (+/- {1:.2%})".format(np.mean(scores), np.std(scores)*2))
print("Execution time: {0:.5} seconds \n".format(end-start))